In [2]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso 
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
from regain.hmm.utils import cross_validation#, cross_validation_auto
import pickle as pkl
from sklearn import cluster
from sklearn.mixture import GaussianMixture



In [3]:
# list of parameters to generate different kind of datasets

states = [2,5,10,15]
dims = [10, 15, 20, 30]
mode_prec = ['regain','regain_random']
mode_mea = ['Normal']#, 'Uniform']

res_all = {}

In [5]:
from sklearn.covariance import empirical_covariance

def compute_precision(X,K,clus):
    
    precisions = []
    N, d = X.shape
    means = np.zeros((K, d))
    
    for i, l in enumerate(np.unique(clus)):
        
        means[i, :] = np.mean(X[np.where(clus == l)[0], :], axis=0)
        emp_cov = empirical_covariance(X - means[i, :],assume_centered=True)
        precisions.append(np.linalg.pinv(emp_cov))
    
    return precisions
    

In [6]:
for st in states:
    for dim in dims:
        for mm in mode_prec:


            print('Case: N_states:',st,',TS dim:',dim,',Prec mode:',mm)

            # Generate a dataset
            dataset = generate_hmm(n_samples=1000,
                                   n_states=st,
                                   n_dim_obs=dim,
                                   mode_precisions=mm,
                                   mode_mean = 'Normal',
                                   transition_type='fixed',
                                   sigma=1)
            # CV hmm_gmm 

            hmm_gmm = HMM_GraphicalLasso(alpha=1,
                                         max_iter = 1000,
                                         n_clusters=st,
                                         verbose=False,
                                         mode='scaled',
                                         warm_restart=True,
                                         repetitions=5,
                                         n_jobs=-1)


            if st == 2:
                alpha_list = np.linspace(35,115 ,20)
            elif st == 5:
                alpha_list = np.linspace(25,55 ,20)
            elif st == 10:
                alpha_list = np.linspace(15,45 ,20)
            elif st == 15:
                alpha_list = np.linspace(5,35 ,20)


            res = cross_validation(hmm_gmm,
                   dataset['data'],
                   params={'alpha':alpha_list,
                           'n_clusters':[st]},
                   n_repetitions=1)

            # define three different models

            hmm_gmm = HMM_GraphicalLasso(alpha=res[0][0], n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=50, n_jobs=-1)
            
            #hmm
            
            hmm = HMM_GraphicalLasso(alpha=0, n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=5, n_jobs=-1)

            
            # gmm 
            gmm = GaussianMixture(n_components=st)

            
            spectral = cluster.SpectralClustering(n_clusters=st,assign_labels="discretize")
            
            # Kmeans
            kmeans = cluster.KMeans(n_clusters=st)
            

            # fit and save results
            try :
                hmm_gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM_GMM')] = results_recap(dataset['states'], 
                                                                   hmm_gmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm_gmm.precisions_)
            except:
                print("HMM-GMM failed")

            
            try :
                hmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM')] = results_recap(dataset['states'], 
                                                                   hmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm.precisions_)
            except:
                print("HMM failed")
            
            try :
                gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'GMM')] = results_recap(dataset['states'], 
                                                              gmm.predict(dataset['data']),
                                                              dataset['thetas'], 
                                                              gmm.precisions_)
            except:
                print("GMM failed")

            
            try :
                spectral.fit(dataset['data'])
                res_all[(st, dim, mm, 'spectral')] = results_recap(dataset['states'], 
                                                                   spectral.labels_,
                                                                   dataset['thetas'], 
                                                                   compute_precision(dataset['data'],
                                                                                     st,
                                                                                     spectral.labels_))
            except:
                print("Spectral failed")


            
            try :
                kmeans.fit(dataset['data'])
                res_all[(st, dim, mm, 'kmeans')] = results_recap(dataset['states'], 
                                                                  kmeans.labels_,
                                                                  dataset['thetas'], 
                                                                  compute_precision(dataset['data'],
                                                                                    st,
                                                                                    kmeans.labels_))
            except:
                print("Kmeans failed")

                

with open('Cluster_methods_comparison2.pickle', 'wb') as f:
    pkl.dump([res_all], f)

  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain



  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain_random



  0%|          | 0/1 [00:00<?, ?it/s]

...

100%|██████████| 20/20 [4:22:22<00:00, 787.11s/it]


Case: N_states: 15 ,TS dim: 30 ,Prec mode: regain


100%|██████████| 20/20 [2:17:56<00:00, 413.81s/it]


Case: N_states: 15 ,TS dim: 30 ,Prec mode: regain_random


100%|██████████| 20/20 [3:32:14<00:00, 636.71s/it]


In [7]:
res_all

{(2, 10, 'regain', 'HMM_GMM'): {'homogeneity [0, 1]': 0.9737104677540755,
  'completeness [0, 1]': 0.9732455175660262,
  'v_measure [0, 1]': 0.9734779371429568,
  'adjusted_mutual_info [0, 1]': 0.9734587205740036,
  'weighted_mean_mcc [-1, 1]': (0.39245027124773957,),
  'max_cluster_mean_mcc[-1,1]': (0.39285714285714285,),
  'weighted_mean_f1 [0, 1]': (0.3636363636363636,),
  'max_cluster_mean_f1[0,1]': (0.3636363636363636,),
  'probabilities_clusters': (array([[0.00632911, 1.        ],
          [0.99367089, 0.        ]]),),
  'max_probabilities_couples': ['Couple: (0, 1), Probability: 1.0, MCC: 0.5, F1_score: 0.3636363636363636',
   'Couple: (1, 0), Probability: 0.9936708860759493, MCC: 0.2857142857142857, F1_score: 0.3636363636363636']},
 (2, 10, 'regain', 'GMM'): {'homogeneity [0, 1]': 0.9367165340782925,
  'completeness [0, 1]': 0.9358712005613077,
  'v_measure [0, 1]': 0.9362936765173234,
  'adjusted_mutual_info [0, 1]': 0.9362475279592943,
  'weighted_mean_mcc [-1, 1]': (0.38882

In [1]:
import pickle

with open('Cluster_methods_comparison.pickle', 'rb') as f:
   [trial] =  pickle.load(f)

In [18]:
trial[(2, 10, 'regain', 'HMM_GMM')]['max_probabilities_couples'][1][12]

'0'